In [13]:
import pandas as pd
import numpy as np

## FX Overlay
## ----------------------------------------------------------------------------------
## 1. Loading data (GDP, Inflation, Futures, 2y yields)
## 2. Building THREE macro signals per region:
##      - Business Cycle   (GDP YoY – Inflation YoY)
##      - Monetary Policy  (– 12m change in 2y yield)
##      - Risk Sentiment   (12m equity futures return)
#       - Here we are missing the FX/ trade data 

## 3. Combines them into a single macro FX score per region & month

## 4. Standardizes scores cross-sectionally (z-scores across regions each month)

## 5. Maping the z-score into a Discrete Hedge Ratio:
##       strong positive -> 0% hedge (expect FX appreciation)
##       mid positive   -> 50% hedge
##       mid negative   -> 75% hedge
##       strong negative -> 100% hedge (expect FX depreciation)

## 6. Signal at t used for t+1 (Shifting the hedge ratio forward by one month)

## 7. Outputs a long-format table: Date, Region, FXScore_z, HedgeRatio_next_month


# --------------------------------------------------------------------------------------------
# 1. Configuration & Data Loading
# --------------------------------------------------------------------------------------------

# Define your local paths here
gdp_path  = r"C:/Users/Sedláček/Documents/UZH/PMP/Macro_momentum/PMP_December_8/Data/GDP_forecasts.csv"      # Expected cols: Date, US, GB, EU, CH, JP, AU, EM
inf_path  = r"C:/Users/Sedláček/Documents/UZH/PMP/Macro_momentum/PMP_December_8/Data/Inflation_forecasts.csv" # Expected cols: Date, US, GB, EU, CH, JP, AU, EM
fut_path  = r"C:/Users/Sedláček/Documents/UZH/PMP/Macro_momentum/PMP_December_8/Data/Futures.xlsx"
y2_path   = r"C:/Users/Sedláček/Documents/UZH/PMP/Macro_momentum/PMP_December_8/Data/2_years_yields.xlsx"

# Load Data
# Note: For CSVs, we use read_csv. For Excel, read_excel.
# We explicitly parse dates and set the index to 'Date' immediately for easier handling.

gdp_forecasts = pd.read_csv(gdp_path, parse_dates=["Date"], index_col="Date")
inf_forecasts = pd.read_csv(inf_path, parse_dates=["Date"], index_col="Date")
fut           = pd.read_excel(fut_path, parse_dates=["Date"], index_col="Date")
y2            = pd.read_excel(y2_path,  parse_dates=["Date"], index_col="Date")


In [ ]:
# --------------------------------------------------------------------------------------------
# 2. Pre-processing & Column Cleaning
# --------------------------------------------------------------------------------------------

# 1. Normalize Column Names (Strip _PX_LAST, etc.)
def clean_columns(df):
    new_cols = []
    for col in df.columns:
        # Split by underscore and take the first part (e.g. US_PX_LAST -> US)
        clean_name = str(col).split('_')[0]
        new_cols.append(clean_name)
    df.columns = new_cols
    return df

y2 = clean_columns(y2)
fut = clean_columns(fut)

# 2. Fix Specific Mismatches (UK -> GB)
# We map "UK" in futures to "GB" to match Yields and GDP data
fut = fut.rename(columns={"UK": "GB"})

# 3. Resample to Month Start
gdp_monthly = gdp_forecasts.asfreq('MS').ffill().sort_index()
inf_monthly = inf_forecasts.asfreq('MS').ffill().sort_index()
fut_monthly = fut.asfreq('MS').ffill().sort_index()
y2_monthly  = y2.asfreq('MS').ffill().sort_index()

# 4. FORCE the Target Regions (The Fix)
# Instead of asking "what exists?", we tell the code "This is what we want."
target_regions = ["US", "GB", "EU", "CH", "JP", "AU", "EM"]

# We reindex all dataframes to this list. 
# If a country is missing in a file (e.g., AU in yields), it creates a column of NaNs.
gdp_monthly = gdp_monthly.reindex(columns=target_regions)
inf_monthly = inf_monthly.reindex(columns=target_regions)
fut_monthly = fut_monthly.reindex(columns=target_regions)
y2_monthly  = y2_monthly.reindex(columns=target_regions)

# Optional: Fill missing columns with 0 if you strictly need numbers now
# (Though leaving them as NaN is usually safer so you know data is missing)
# gdp_monthly = gdp_monthly.fillna(0) 

# Update the variable name for the next steps
valid_regions = target_regions

print(f"Target Regions set to: {valid_regions}")

Target Regions set to: ['US', 'GB', 'EU', 'CH', 'JP', 'AU', 'EM']
Columns missing in files have been created as NaN (empty).


In [22]:
# --------------------------------------------------------------------------------------------
# 3. Signal Generation (Raw)
# --------------------------------------------------------------------------------------------
# Note: We use .diff(12) or .pct_change(12). If data is shorter than 12 months, this creates NaNs.

# A. Business Cycle (BC)
gdp_change = gdp_monthly[valid_regions].diff(12) 
inf_change = inf_monthly[valid_regions].diff(12) 
bc_raw = (gdp_change + inf_change) / 2.0

# B. Monetary Policy (MP)
mp_raw = y2_monthly[valid_regions].diff(12)

# C. Risk Sentiment (RS)
rs_raw = fut_monthly[valid_regions].pct_change(12)

C:\Users\Sedláček\AppData\Local\Temp\ipykernel_23468\1289828401.py:15: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rs_raw = fut_monthly[valid_regions].pct_change(12)


In [ ]:
# --------------------------------------------------------------------------------------------
# 4. Longitudinal Standardization
# --------------------------------------------------------------------------------------------

def calculate_historic_zscore(df, min_periods=12):
    # Lowered min_periods to 12 to ensure you get data faster
    rolling_mean = df.expanding(min_periods=min_periods).mean()
    rolling_std  = df.expanding(min_periods=min_periods).std()
    z_score = (df - rolling_mean) / rolling_std.replace(0, np.nan)
    return z_score.clip(-3, 3)

bc_z = calculate_historic_zscore(bc_raw)
mp_z = calculate_historic_zscore(mp_raw)
rs_z = calculate_historic_zscore(rs_raw)

In [23]:
# --------------------------------------------------------------------------------------------
# 5. Composite Signal Construction (ROBUST MEAN FIX)
# --------------------------------------------------------------------------------------------
# Instead of (A+B+C)/3, we sum available values and divide by the count of available values.
# This prevents one missing country indicator from killing the whole score.

# 1. Sum up all standardized signals, treating NaN as 0
sum_df = bc_z.fillna(0) + mp_z.fillna(0) + rs_z.fillna(0)

# 2. Count how many valid signals exist for each cell
count_df = bc_z.notna().astype(int) + mp_z.notna().astype(int) + rs_z.notna().astype(int)

# 3. Divide Sum by Count (replacing 0 count with NaN to avoid division by zero)
composite_signal = sum_df / count_df.replace(0, np.nan)

In [24]:
# --------------------------------------------------------------------------------------------
# 6. Cross-Sectional Ranking
# --------------------------------------------------------------------------------------------

def cross_sectional_zscore(df):
    return df.sub(df.mean(axis=1), axis=0).div(df.std(axis=1), axis=0)

final_z_score = cross_sectional_zscore(composite_signal)

In [25]:
# --------------------------------------------------------------------------------------------
# 7. Hedge Ratio Mapping
# --------------------------------------------------------------------------------------------

def get_hedge_ratio(z_val):
    if pd.isna(z_val): return np.nan
    if z_val > 1.0: return 0.0     # Strong Bullish -> No Hedge
    if z_val > 0.0: return 0.50    # Mild Bullish -> 50% Hedge
    if z_val > -1.0: return 0.75   # Mild Bearish -> 75% Hedge
    return 1.0                     # Strong Bearish -> 100% Hedge

hedge_ratios = final_z_score.applymap(get_hedge_ratio)
hedge_ratios_next_month = hedge_ratios.shift(1)

C:\Users\Sedláček\AppData\Local\Temp\ipykernel_23468\3598253578.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  hedge_ratios = final_z_score.applymap(get_hedge_ratio)


In [27]:
# --------------------------------------------------------------------------------------------
# 8. Export (ROBUST EXPORT FIX)
# --------------------------------------------------------------------------------------------

# We use dropna=False to keep the dates even if data is missing.
# We also use a safer join method to combine the Score and Ratio.

# Stack the hedge ratios
df_hedge = hedge_ratios_next_month.stack(dropna=False).to_frame(name='Hedge_Ratio_Next_Month')

# Stack the raw Z-scores (shifted to match the hedge ratio timing)
df_score = final_z_score.shift(1).stack(dropna=False).to_frame(name='FX_Score_Z')

# Merge them based on index (Date, Region)
output = df_hedge.join(df_score)

# Reset index to turn Date/Region into columns
output = output.reset_index()
output.columns = ['Date', 'Region', 'Hedge_Ratio_Next_Month', 'FX_Score_Z']

# Filter out rows where BOTH are NaN (truly empty dates), but keep rows with partial data
output = output.dropna(subset=['Hedge_Ratio_Next_Month', 'FX_Score_Z'], how='all')

print("Output Tail:")
print(output.tail(30))

#output.to_csv("FX_Hedge_Signals_Final.csv", index=False)

Output Tail:
           Date Region  Hedge_Ratio_Next_Month  FX_Score_Z
4655 2025-06-01     US                    0.75   -0.299579
4656 2025-06-01     GB                    0.50    0.289773
4657 2025-06-01     EU                    0.75   -0.124443
4658 2025-06-01     CH                    0.50    0.146971
4659 2025-06-01     JP                    0.00    1.541075
4660 2025-06-01     AU                    1.00   -1.553797
4662 2025-07-01     US                    0.75   -0.325393
4663 2025-07-01     GB                    0.50    0.310882
4664 2025-07-01     EU                    0.75   -0.095166
4665 2025-07-01     CH                    0.50    0.264220
4666 2025-07-01     JP                    0.00    1.456785
4667 2025-07-01     AU                    1.00   -1.611328
4669 2025-08-01     US                    0.50    0.078545
4670 2025-08-01     GB                    0.00    1.181756
4671 2025-08-01     EU                    0.75   -0.353109
4672 2025-08-01     CH                    1

C:\Users\Sedláček\AppData\Local\Temp\ipykernel_23468\3645682200.py:9: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_hedge = hedge_ratios_next_month.stack(dropna=False).to_frame(name='Hedge_Ratio_Next_Month')
C:\Users\Sedláček\AppData\Local\Temp\ipykernel_23468\3645682200.py:12: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_score = final_z_score.shift(1).stack(dropna=False).to_frame(name='FX_Score_Z')


In [8]:
bc_signal = bc_signal.sort_values("Date", ascending=False)#.reset_index(drop=True)
mp_signal = mp_signal.sort_values("Date", ascending=False)
rs_signal = rs_signal.sort_values("Date", ascending=False)

print(bc_signal)
print(mp_signal)
print(rs_signal)

             US   GB   EU   JP   CH   EM   AU
Date                                         
2025-10-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2025-09-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2025-08-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2025-07-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2025-06-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0
...         ...  ...  ...  ...  ...  ...  ...
1998-01-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1997-12-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1997-11-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1997-10-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1997-09-01  NaN  NaN  NaN  NaN  NaN  NaN  NaN

[338 rows x 7 columns]
                US     GB     EU     JP     CH      EM  AU
Date                                                      
2025-10-01  0.5966  0.666  0.313 -0.479  0.528  0.4895 NaN
2025-09-01  0.0328 -0.002  0.049 -0.552  0.532  0.1547 NaN
2025-08-01  0.2998  0.166  0.451 -0.497  0.693  0.3335 NaN
2025-07-01  0.3004 -0.035  0.567 -0.367  0.742  0.5227 NaN
2025-06-01  1.0343  0.40